In [ ]:
#CS391Project
#Charlie, Ben, Jenna

In [1]:
import pandas as pd
import datetime as dt
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt   

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Reading in CSV Files
df = pd.read_csv('CrashMaryland.csv')
dfW = pd.read_csv('Woodbine_2017_Crash.csv')

In [3]:
#Setting cut-off dates for dataframe
date2017 = dt.datetime(2017, 1, 1, 0,0,0)
date2018 = dt.datetime(2017, 12, 31, 0,0,0)

In [4]:
#start of data preprocessing
df['date'] = pd.to_datetime(df['Crash Date/Time'])

dfW['Date/Time'] = dfW['Date/Time'].map(lambda x: str(x)[:-3]) 
dfW['date'] = pd.to_datetime(dfW['Date/Time'])

for i in range(len(df['date'])):
    df['date'].loc[i]= df['date'].loc[i].replace(minute=0)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
#creating dataframe with dates only from 2017
df201718 = df.loc[df['date'] >= date2017]
mask = (df201718['date'] >= date2017) & (df201718['date'] <= date2018)
df2017 = df201718.loc[mask] 

In [7]:
#Converting some useful catagorical features
df2017['Weather'] = df2017['Weather'].astype('category')  
df2017['weatherCat'] = df2017['Weather'].cat.codes

df2017['Light'] = df2017['Light'].astype('category')  
df2017['lightCat'] = df2017['Light'].cat.codes

df2017['Surface Condition'] = df2017['Surface Condition'].astype('category')  
df2017['surfaceConditionCat'] = df2017['Surface Condition'].cat.codes

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [8]:
#Merging weather to 2017 crash data
df2017W= pd.merge(df2017, dfW, on='date')

In [9]:
#Dataframe with number of crashes at certain hour of day
crashNum = df2017W['date'].value_counts().to_frame()
dfCW = dfW
dfCW.set_index('date', inplace = True)

In [10]:
#Joining number of crashes with weather table
dfCW = dfCW.join(crashNum, how = 'outer')
dfCW.fillna(0, inplace = True)

In [11]:
#creating column to show if a crash occured at a certain time
dfCW['crashOccur'] = dfCW['date'] != 0
dfCW.replace({True: 1, False: 0}, inplace=True)

dfCW['hours'] = dfCW.index.hour
dfCW.drop('Date/Time', axis = 1, inplace = True)

dfCW.to_csv('crashWeather.csv')

In [12]:
#Training and Splitting Data
X = dfCW[['Air Temp (℉)', 'Precip (inches)', 'Wind Spd (mph)', 'RH (%)', 'Leaf Wetness (minutes)', 'hours']]
Y = dfCW['crashOccur']

Xl = dfCW[['Air Temp (℉)']]
Yl = dfCW['date']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [13]:
#Random Forest Classifier 
clf=RandomForestClassifier()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print("Accuracy of Forest:",metrics.accuracy_score(y_test, y_pred))
feature_importances = pd.DataFrame(clf.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

Accuracy of Forest: 0.6727549467275494
                        importance
Air Temp (℉)              0.296858
hours                     0.253201
RH (%)                    0.209857
Wind Spd (mph)            0.198637
Leaf Wetness (minutes)    0.024489
Precip (inches)           0.016959


In [14]:
#logistic regression
logit_model=sm.Logit(Y,X)
result=logit_model.fit()
print(result.summary())


logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
kfold = model_selection.KFold(n_splits=10, random_state=7)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

print(classification_report(y_test, y_pred))

Optimization terminated successfully.
         Current function value: 0.612314
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             crashOccur   No. Observations:                 8760
Model:                          Logit   Df Residuals:                     8754
Method:                           MLE   Df Model:                            5
Date:                Thu, 08 Nov 2018   Pseudo R-squ.:                 0.05406
Time:                        18:52:52   Log-Likelihood:                -5363.9
converged:                       True   LL-Null:                       -5670.4
                                        LLR p-value:                3.074e-130
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Air Temp (℉)               0.0082      0.001      6.900      0.000       0.006      

In [15]:
#BernoulliNB
clf=BernoulliNB()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print("Accuracy of BernoulliNB:",metrics.accuracy_score(y_test, y_pred))

Accuracy of BernoulliNB: 0.6464992389649924


In [16]:
#GaussianNB
clf2=GaussianNB()
clf2.fit(X_train,y_train)
y_pred=clf2.predict(X_test)
print("Accuracy of GaussianNB:",metrics.accuracy_score(y_test, y_pred))

Accuracy of GaussianNB: 0.6757990867579908


In [17]:
#MLP Classifier
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

mlp = MLPClassifier()
mlp.fit(X_train,y_train)

predictions = mlp.predict(X_test)
predictTrain = mlp.predict(X_train)
probs =  mlp.predict_proba(X_test)


print("Accuracy of MLPClassifier training data:",metrics.accuracy_score(y_train, predictTrain))
print("Accuracy of MLPClassifier test data:",metrics.accuracy_score(y_test, predictions))

print('Train Report')
print(classification_report(y_train, predictTrain))

print('Test Report')
print(classification_report(y_test,predictions))

cm = confusion_matrix(y_test, predictions)
print(cm)

Accuracy of MLPClassifier training data: 0.726027397260274
Accuracy of MLPClassifier test data: 0.7264079147640792
Train Report
             precision    recall  f1-score   support

          0       0.64      0.47      0.54      2107
          1       0.76      0.86      0.80      4025

avg / total       0.72      0.73      0.71      6132

Test Report
             precision    recall  f1-score   support

          0       0.68      0.47      0.56       957
          1       0.74      0.87      0.80      1671

avg / total       0.72      0.73      0.71      2628

[[ 453  504]
 [ 215 1456]]
